In [ ]:
from icu_experiments.load_data import load_data_for_prediction
from icu_experiments.preprocessing import make_feature_preprocessing, make_anchor_preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor, Booster
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from ivmodels import AnchorRegression

outcome = "hr"

Xy = load_data_for_prediction(["eicu"], outcome=outcome, log_transform=True)
Xy_train = Xy['eicu']['train']
Xy_test = Xy['eicu']['test']

Xy_new = load_data_for_prediction(['hirid'], outcome=outcome, log_transform=True)
Xy_test_new = Xy_new['hirid']['train']
Xy_tuning_data = Xy_new['hirid']['test']

preprocessing_steps = make_feature_preprocessing(missing_indicator=True)
preprocessor = ColumnTransformer(transformers=preprocessing_steps).set_output(transform="pandas") # Allow to preprocess subbsets of data differently

anchor_columns = ['hospital_id']
anchor_preprocessing_steps = make_anchor_preprocessing(anchor_columns)
anchor_preprocessor = ColumnTransformer(
        anchor_preprocessing_steps + preprocessing_steps
        #preprocessing_steps
    ).set_output(transform="pandas")

p1 = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', LGBMRegressor())
])
p2 = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', LGBMRegressor())
])
p3 = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', LinearRegression())
])
p4 = Pipeline(steps=[
    ('preprocessing', anchor_preprocessor),
    ('model', AnchorRegression())
])

param_grid_lgbm = {
    'model__boosting_type': ['gbdt'],  # Set the boosting type for LightGBM
    'model__num_leaves': [15, 31],
    'model__subsample': [0.8, 1.0],
    'model__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
    'model__n_estimators': [50, 100, 800]
}
param_grid_rf = {
    'model__boosting_type': ['rf'],  # Set the boosting type for LightGBM
    'model__num_leaves': [15, 31],
    'model__subsample': [0.8, 1.0],
    'model__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
    'model__n_estimators': [50, 100, 800]
}
param_grid_anchor = {
    'instrument_regex': ["anchor"],
    'gamma': [1, 3.16, 10, 31.6, 100, 316, 1000, 3162, 10000],
    'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1]
}

### Compare Performance from Training to Target Data - Parameters chosen via GridCV on Training Data

In [3]:
search = GridSearchCV(p1, param_grid_lgbm)
search.fit(Xy_train, Xy_train['outcome'])
print("Best parameter (CV score=%0.3f):" % search.best_score_)
grid_search_params_lgbm = search.best_params_
print(search.best_params_)

search = GridSearchCV(p2, param_grid_rf)
search.fit(Xy_train, Xy_train['outcome'])
print("Best parameter (CV score=%0.3f):" % search.best_score_)
grid_search_params_rf = search.best_params_
print(search.best_params_)

p1.set_params(**grid_search_params_lgbm)
p1.fit(Xy_train, Xy_train['outcome'])
p2.set_params(**grid_search_params_rf)
p2.fit(Xy_train, Xy_train['outcome'])

mse_grid_lgbm = mean_squared_error(Xy_test_new['outcome'], p1.predict(Xy_test_new))
mse_grid_rf = mean_squared_error(Xy_test_new['outcome'], p2.predict(Xy_test_new))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005777 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12253
[LightGBM] [Info] Number of data points in the train set: 58145, number of used features: 99
[LightGBM] [Info] Start training from score 84.924568
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017396 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12228
[LightGBM] [Info] Number of data points in the train set: 58145, number of used features: 99
[LightGBM] [Info] Start training from score 84.928896
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020333 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12230
[LightGBM] [Info] Number of data points in the trai

### Compare Performance from Training to Target Data - Parameters chosen via Evaluation on Target Data

- Train Data with different parameters on Training set
- Evaluate Train Data on fine tuning data from target set 
- choose the best performing parameters
- do this for all possible n from the fine tuning data set

In [ ]:
def find_best_parameters(Xy_train, Xy_tuning_data, p, param_grid):
    # Initialize a list to store the best parameters and MSE for each n
    results_for_n = []

    for n in [25, 50, 100, 200, 400, 800, 1600]:
        # Initialize variables to keep track of the best parameters and MSE for the current n
        best_params = None
        best_mse = float('inf')  # Initialize with a large value

        # Iterate over all possible combinations of hyperparameters
        for param_set in itertools.product(*param_grid.values()):
            param_dict = dict(zip(param_grid.keys(), param_set))

            # Create and train the LightGBM model
            params = {
                **param_dict  # Include other relevant parameters
            }
            p.set_params(**{'model__' + key: value for key, value in params.items()})
            p.fit(Xy_train, Xy_train['outcome'])

            # Make predictions on the subset of data
            y_pred = p.predict(Xy_tuning_data.head(n))

            # Calculate mean squared error for this parameter set
            mse = mean_squared_error(Xy_tuning_data['outcome'].head(n), y_pred)

            # Check if this MSE is better than the current best for the current n
            if mse < best_mse:
                best_mse = mse
                best_params = param_dict

        # Store the best parameters and MSE for the current n in the list
        results_for_n.append({'n': n, 'best_params': best_params, 'best_mse': best_mse})

    return results_for_n

results_p1 = find_best_parameters(Xy_train, Xy_tuning_data, p1, param_grid_lgbm)
results_p2 = find_best_parameters(Xy_train, Xy_tuning_data, p2, param_grid_rf)
results_p4 = find_best_parameters(Xy_train, Xy_tuning_data, p4, param_grid_anchor)

def calculate_mse(X_train, y_train, X_test, y_test, p, results, boosting_type=None, is_anchor=False):
    mse_for_n = []
    i = 0
    for n in [25, 50, 100, 200, 400, 800, 1600]:
        params = {f'model__{key}': value for key, value in results[i]['best_params'].items()}
        if not is_anchor: 
            params['model__boosting_type'] = boosting_type
        p.set_params(**params)
        p.fit(X_train, y_train)
        y_pred = p.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_for_n.append({'n': n, 'mse': mse})
        i += 1
    return mse_for_n

mse_eicu_to_hirid_p1 = calculate_mse(Xy_train, Xy_train['outcome'], Xy_test_new, Xy_test_new['outcome'], p1, results_p1, 'gbdt')
mse_eicu_to_hirid_p2 = calculate_mse(Xy_train, Xy_train['outcome'], Xy_test_new, Xy_test_new['outcome'], p2, results_p2, 'rf')
mse_eicu_to_hirid_p4 = calculate_mse(Xy_train, Xy_train['outcome'], Xy_test_new, Xy_test_new['outcome'], p4, results_p4, is_anchor=True)

# OLS MSE Calculation
p3.fit(Xy_train, Xy_train['outcome'])
mse_eicu_to_hirid_p3 = mean_squared_error(Xy_test_new['outcome'], p3.predict(Xy_test_new, Xy_test_new['outcome']))
mse_eicu_to_hirid_dummy_prediction = mean_squared_error(Xy_test_new['outcome'], np.full_like(Xy_test_new['outcome'],Xy_train[outcome].mean()))

In [ ]:
n = [25, 50, 100, 200, 400, 800, 1600]
plt.plot(n, ([item['mse'] for item in mse_eicu_to_hirid_p1]), marker='o', linestyle='-', label = 'LGBM')
plt.plot(n, [item['mse'] for item in mse_eicu_to_hirid_p2], marker='o', linestyle='-', label = 'RF')
plt.plot(n, [item['mse'] for item in mse_eicu_to_hirid_p4], marker='o', linestyle='-', label = 'Anchor')
plt.axhline(y=mse_eicu_to_hirid_p3, color='red', linestyle='-', label='OLS Baseline')
plt.axhline(y=mse_grid_lgbm, color='green', linestyle='-', label='LGBM Baseline')
plt.axhline(y=mse_grid_rf, color='purple', linestyle='-', label='RF Baseline')
#plt.axhline(y = mean_squared_error(Xy_test_new['outcome'], np.full_like(Xy_test_new['outcome'],Xy_train[outcome].mean())), color = 'black', label='Train Average')
plt.title('Parameters for Model chosen with evaluation on n Data Points from Target Distribution')
plt.xlabel('Number of Data Points (n)')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.grid(True)
plt.show()

The hyperparameters were chosen such that they minimize the the mse on the fine-tuning data set of the target distribution. We have n = 25, 50, 100, 200, 400, 800 and 1600 datapoints from the target distribution. In an inital step, we chose randomly 1600 data points from the target data and called it Xy_tuning_data - they are not part of the final evaulation data (i.e. the plotted mse after training). The hyperparameters were chosen from: 

param_grid_lgbm = { \
    'boosting_type': ['gbdt'], \
    'num_leaves': [15, 31], \
    'subsample': [0.8, 1.0], \ 
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],  # Increase the range of learning rates \
    'n_estimators': [50, 100, 800]  # Increase the range of estimators\
} \

param_grid_rf = {\
    'boosting_type': ['rf'],\
    'num_leaves': [15, 31],\
    'subsample': [0.8, 1.0],\
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],  # Increase the range of learning rates\
    'n_estimators': [50, 100, 800]  # Increase the range of estimators\
}\

param_grid_anchor = {\
    'instrument_regex': ["anchor"],\
    'gamma': [1, 3.16, 10, 31.6, 100, 316, 1000, 3162, 10000],\
    'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1]\
}


The parameters for evaluation were chosen as follows:
1. For each combination of the parameters train the model on the training data
2. Calculate the mse on the fine-tuning data from the trainings distribution 
3. For each n, choose the parameter combination such that it minimizes the mse on the fine-tuning data

The LGBM-pipeline is called p1, the RF-pipeline is called p2, the OLS-pipeline is called p3, the Anchor-pipeline is called p4.

For OLS we proceed different. Here we simply train the model on the trainings data and evaluate it on the target data. 

In a successive step, we perform the same parameter selection process but on the trainings data and calculate the mse on the target data. The performance is denoted in the plot by the model + Baseline. Surprisingly, none of the models was able to beat the Baselines. Only LGBM was able for n = 1600 to accomplish identical precision as its baseline. It seems like LGBM can already identify the same important hyperparameters when only having access to 1600 tuning data points. Already after just having 50 data points available, anchor is able to beat OLS. The predictive performance of the RF seems to be independent of the choosen parameters. The choosen parameters seem to have high influence on the predictive performance of LGBM. After only having 200 data points available, LGBM is able to reduce its mse by 8%. 

This may be caused by the lack of available hyperparameters. It would be interisting to see if the models are able to beat its baseline when allow more possibilities. A possible follow up question is if the predictive perfromance improves when n=2000 (Hypothesis: Yes, since the prediction has more accurate data)

All models were able to beat the average-prediction of the trainings data by a lot (MSE = 283.59190335035487)

#### The parameters: 
'best_mse' is the mse that was reached with this parameter combination on the target data 

for p1 / LGBM: 
[{'n': 25,
  'best_params': {'boosting_type': 'gbdt',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.2,
   'n_estimators': 800},
  'best_mse': 113.7853669744457},
 {'n': 50,
  'best_params': {'boosting_type': 'gbdt',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.2,
   'n_estimators': 800},
  'best_mse': 151.03257816415527},
 {'n': 100,
  'best_params': {'boosting_type': 'gbdt',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.2,
   'n_estimators': 800},
  'best_mse': 147.50302668110243},
 {'n': 200,
  'best_params': {'boosting_type': 'gbdt',
   'num_leaves': 31,
   'subsample': 0.8,
   'learning_rate': 0.1,
   'n_estimators': 100},
  'best_mse': 159.93871260736046},
 {'n': 400,
  'best_params': {'boosting_type': 'gbdt',
   'num_leaves': 31,
   'subsample': 0.8,
   'learning_rate': 0.1,
   'n_estimators': 100},
  'best_mse': 153.88405924738646},
 {'n': 800,
  'best_params': {'boosting_type': 'gbdt',
   'num_leaves': 31,
   'subsample': 0.8,
   'learning_rate': 0.1,
   'n_estimators': 100},
  'best_mse': 148.97829215300928},
 {'n': 1600,
  'best_params': {'boosting_type': 'gbdt',
   'num_leaves': 31,
   'subsample': 0.8,
   'learning_rate': 0.01,
   'n_estimators': 800},
  'best_mse': 163.99952271689486}]

  For p2 / RF: 
  [{'n': 25,
  'best_params': {'boosting_type': 'rf',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.001,
   'n_estimators': 800},
  'best_mse': 140.62296079918235},
 {'n': 50,
  'best_params': {'boosting_type': 'rf',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.001,
   'n_estimators': 800},
  'best_mse': 169.23772353266452},
 {'n': 100,
  'best_params': {'boosting_type': 'rf',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.001,
   'n_estimators': 800},
  'best_mse': 156.54175652388255},
 {'n': 200,
  'best_params': {'boosting_type': 'rf',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.001,
   'n_estimators': 800},
  'best_mse': 170.28568523948127},
 {'n': 400,
  'best_params': {'boosting_type': 'rf',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.001,
   'n_estimators': 800},
  'best_mse': 159.52549273668203},
 {'n': 800,
  'best_params': {'boosting_type': 'rf',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.001,
   'n_estimators': 800},
  'best_mse': 154.98743617760542},
 {'n': 1600,
  'best_params': {'boosting_type': 'rf',
   'num_leaves': 15,
   'subsample': 0.8,
   'learning_rate': 0.001,
   'n_estimators': 800},
  'best_mse': 171.13541258639327}]

  For p3 / OLS: 
{'hirid': 'OLS': {'Test Error': 159.65984526272683}} 

For p4 / Anchor: 
[{'n': 25,
  'best_params': {'instrument_regex': 'anchor', 'gamma': 1, 'alpha': 0.1},
  'best_mse': 145.0284017501739},
 {'n': 50,
  'best_params': {'instrument_regex': 'anchor', 'gamma': 1, 'alpha': 0.001},
  'best_mse': 166.65092062151024},
 {'n': 100,
  'best_params': {'instrument_regex': 'anchor', 'gamma': 3.16, 'alpha': 0.01},
  'best_mse': 152.59928450652117},
 {'n': 200,
  'best_params': {'instrument_regex': 'anchor', 'gamma': 1, 'alpha': 1e-05},
  'best_mse': 165.26237755409892},
 {'n': 400,
  'best_params': {'instrument_regex': 'anchor', 'gamma': 1, 'alpha': 1e-05},
  'best_mse': 155.53044386121334},
 {'n': 800,
  'best_params': {'instrument_regex': 'anchor', 'gamma': 1, 'alpha': 1e-05},
  'best_mse': 151.55849601840572},
 {'n': 1600,
  'best_params': {'instrument_regex': 'anchor', 'gamma': 1, 'alpha': 0.01},
  'best_mse': 165.51230906082435}]



#### Evaluation MSE

For p1 / LGBM: 
[{'n': 25, 'mse': 172.77225379649616},
 {'n': 50, 'mse': 172.77225379649616},
 {'n': 100, 'mse': 172.77225379649616},
 {'n': 200, 'mse': 158.45334348900073},
 {'n': 400, 'mse': 158.45334348900073},
 {'n': 800, 'mse': 158.45334348900073},
 {'n': 1600, 'mse': 157.66217932606298}]

 For p2 / RF: 
 [{'n': 25, 'mse': 163.39728426435462},
 {'n': 50, 'mse': 163.39728426435462},
 {'n': 100, 'mse': 163.39728426435462},
 {'n': 200, 'mse': 163.39728426435462},
 {'n': 400, 'mse': 163.39728426435462},
 {'n': 800, 'mse': 163.39728426435462},
 {'n': 1600, 'mse': 163.39728426435462}]

For p4 / Anchor: 
[{'n': 25, 'mse': 159.97760862179274},
 {'n': 50, 'mse': 159.47863383193007},
 {'n': 100, 'mse': 158.96353854009493},
 {'n': 200, 'mse': 159.6254589611603},
 {'n': 400, 'mse': 159.6254589611603},
 {'n': 800, 'mse': 159.6254589611603},
 {'n': 1600, 'mse': 158.93096747576763}]

#### Baseline MSE

Best parameter (CV score=0.430):
{'model__boosting_type': 'gbdt', 'model__learning_rate': 0.01, 'model__n_estimators': 800, 'model__num_leaves': 31, 'model__subsample': 0.8}


Best parameter (CV score=0.408):
{'model__boosting_type': 'rf', 'model__learning_rate': 0.001, 'model__n_estimators': 800, 'model__num_leaves': 31, 'model__subsample': 0.8}


### More Parameters

In [ ]:
param_grid_lgbm = {
    'model__boosting_type': ['gbdt'],  # Set the boosting type for LightGBM
    'model__subsample': [0.6, 0.8, 1.0],
    'model__learning_rate': [0.001, 0.01, 0.05 0.1, 0.2, 0.3],
    'model__n_estimators': [50, 100, 800, 3000]
}
param_grid_rf = {
    'model__boosting_type': ['rf'],  # Set the boosting type for LightGBM
    'model__subsample': [0.6, 0.8, 1.0],
    'model__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2, 0.3],
    'model__n_estimators': [50, 100, 800, 3000]
}

results_p1_new = find_best_parameters(Xy_train, Xy_tuning_data, p1, param_grid_lgbm)
results_p2_new = find_best_parameters(Xy_train, Xy_tuning_data, p2, param_grid_rf)

mse_eicu_to_hirid_p1_new = calculate_mse(Xy_train, Xy_train['outcome'], Xy_test_new, Xy_test_new['outcome'], p1, results_p1_new, 'gbdt')
mse_eicu_to_hirid_p2_new = calculate_mse(Xy_train, Xy_train['outcome'], Xy_test_new, Xy_test_new['outcome'], p2, results_p2_new, 'rf')

Surprisingly, there is no predictive improvement if we allow more parameters in the selection process:

For p1 / LGBM:
[{'n': 25, 'mse': 172.77225379649616},
 {'n': 50, 'mse': 172.77225379649616},
 {'n': 100, 'mse': 172.77225379649616},
 {'n': 200, 'mse': 158.45334348900073},
 {'n': 400, 'mse': 158.45334348900073},
 {'n': 800, 'mse': 158.45334348900073},
 {'n': 1600, 'mse': 157.66217932606298}]

 For p2 / RF: 
 [{'n': 25, 'mse': 163.39728426435462},
 {'n': 50, 'mse': 163.39728426435462},
 {'n': 100, 'mse': 163.39728426435462},
 {'n': 200, 'mse': 163.39728426435462},
 {'n': 400, 'mse': 163.39728426435462},
 {'n': 800, 'mse': 163.39728426435462},
 {'n': 1600, 'mse': 163.39728426435462}]

# ToDo:

- Malte Fragen beantworten: 
    - Sowohl RF wie LGBM mit tuning auf target distr. data sind schlechter (nie besser) als die Baselines, egal wie gross “n” ist. Wieso?
    - Dein MSE der OLS baseline eICU -> MIMIC III ist signifikant besser als das was ich in dem pdf das ich dir mal geschickt hatte habe (~175). Was machst du anders?
- Andere Target Distr. anschauen
- .md schön schreiben
- Refit implementieren und anschauen
- Customized Anchor implementieren